# OpenCV示例：色彩空间转换


In [ ]:

import cv2
import numpy as np
import sys
import os

# 添加utils目录到路径（兼容中文路径和Jupyter）
if "__file__" in globals():
    utils_path = os.path.join(os.path.dirname(__file__), "..", "utils")
else:
    utils_path = os.path.abspath(os.path.join("..", "utils"))
if os.path.exists(utils_path):
    sys.path.append(utils_path)
    from io_helpers import imread_chinese, get_image_path

# 读取图像（使用中文路径兼容函数）
img_path = get_image_path('sample-images/colored-objects/red-apple.jpg')
if img_path is None:
    print("错误：无法找到图片文件！")
    exit()
img = imread_chinese(img_path)

if img is None:
    print("无法读取图片！")
    print("提示：请确保 assets/sample-images/colored-objects/red-apple.jpg 存在")
    # 创建测试图像
    img = np.zeros((300, 300, 3), dtype=np.uint8)
    img[:] = (0, 0, 255)  # 红色
    cv2.circle(img, (150, 150), 100, (0, 255, 0), -1)  # 绿色圆

print("色彩空间转换")



In [ ]:
print("\n1. BGR与RGB转换")

# BGR转RGB（用于matplotlib显示）
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
print("[OK] BGR → RGB")

# 使用索引反转（快速方法）
img_rgb2 = img[:, :, ::-1]
print("[OK] BGR → RGB (索引反转)")



In [ ]:
print("\n2. BGR转灰度")

# 方法1：使用cvtColor
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# 方法2：读取时直接转灰度
img_gray_read = imread_chinese(img_path, cv2.IMREAD_GRAYSCALE)

print(f"彩色图形状: {img.shape}")
print(f"灰度图形状: {gray.shape}")



In [ ]:
print("\n3. BGR转HSV")

hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

print(f"HSV图像形状: {hsv.shape}")
print("HSV范围:")
print("  - H (色调): 0-180")
print("  - S (饱和度): 0-255")
print("  - V (明度): 0-255")

# 显示HSV各个通道
h, s, v = cv2.split(hsv)



In [ ]:
print("\n4. 颜色提取（红色）")

# 定义红色的HSV范围（注意：红色跨越HSV的0和180）
lower_red1 = np.array([0, 100, 100])
upper_red1 = np.array([10, 255, 255])

lower_red2 = np.array([170, 100, 100])
upper_red2 = np.array([180, 255, 255])

# 创建掩膜
mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
mask2 = cv2.inRange(hsv, lower_red2, upper_red2)

# 合并掩膜
mask = cv2.bitwise_or(mask1, mask2)

# 应用掩膜
result = cv2.bitwise_and(img, img, mask=mask)

# 统计红色像素数量
red_pixels = cv2.countNonZero(mask)
print(f"检测到红色像素: {red_pixels} 个")



In [ ]:
print("\n5. 提取不同颜色")

# 定义颜色字典
colors = {
    'red': ([0, 100, 100], [10, 255, 255]),
    'green': ([35, 100, 100], [85, 255, 255]),
    'blue': ([85, 100, 100], [135, 255, 255])
}

for color_name, (lower, upper) in colors.items():
    lower = np.array(lower)
    upper = np.array(upper)
    mask = cv2.inRange(hsv, lower, upper)
    pixels = cv2.countNonZero(mask)
    print(f"{color_name.capitalize()}: {pixels} 像素")



In [ ]:
print("\n显示结果...")
print("按任意键关闭所有窗口...")

# 创建对比图
# 第一行：原图、灰度、掩膜
row1 = np.hstack([img,
                  cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR),
                  cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)])

# 第二行：H通道、S通道、V通道
row2 = np.hstack([cv2.cvtColor(h, cv2.COLOR_GRAY2BGR),
                  cv2.cvtColor(s, cv2.COLOR_GRAY2BGR),
                  cv2.cvtColor(v, cv2.COLOR_GRAY2BGR)])

# 第三行：HSV、颜色提取结果
row3 = np.hstack([cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR), result])

# 垂直拼接
result_all = np.vstack([row1, row2, row3])

# 调整显示大小
result_display = cv2.resize(result_all, None, fx=0.6, fy=0.6)

cv2.imshow('Color Space Conversion', result_display)
cv2.waitKey(0)
cv2.destroyAllWindows()



In [ ]:
print("\n保存结果 (已禁用)...")

# 保存功能已禁用，如需保存请取消以下注释
# cv2.imwrite('gray.jpg', gray)
# cv2.imwrite('hsv.jpg', hsv)
# cv2.imwrite('red_mask.jpg', mask)
# cv2.imwrite('red_extracted.jpg', result)
print("[提示] 图片保存功能已禁用，避免生成多余文件")

print("\n程序运行完成！")
